In [3]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [4]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


In [9]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
data = np.load('proyecto_training_data.npy')
column_names = ["SalePrice","OverallQual","1stFlrSF","TotRmsAbrGrd","YearBuilt","LotFrontage"]
data_df = pd.DataFrame(data, columns=column_names)

In [10]:
data_df

,SalePrice,OverallQual,1stFlrSF,TotRmsAbrGrd,YearBuilt,LotFrontage
0,208500.0,7.0,856.0,8.0,2003.0,65.0
1,181500.0,6.0,1262.0,6.0,1976.0,80.0
2,223500.0,7.0,920.0,6.0,2001.0,68.0
3,140000.0,7.0,961.0,7.0,1915.0,60.0
4,250000.0,8.0,1145.0,9.0,2000.0,84.0
...,...,...,...,...,...,...
1455,175000.0,6.0,953.0,7.0,1999.0,62.0
1456,210000.0,6.0,2073.0,7.0,1978.0,85.0
1457,266500.0,7.0,1188.0,9.0,1941.0,66.0
1458,142125.0,5.0,1078.0,5.0,1950.0,68.0


In [28]:
correlacion = np.corrcoef(data,rowvar=False)[0,:]
pd.DataFrame(correlacion[1:].reshape(1,5),columns=column_names[1:])

,OverallQual,1stFlrSF,TotRmsAbrGrd,YearBuilt,LotFrontage
0,0.790982,0.605852,0.533723,0.522897,NaN


### Variable con mayor correlacion "OverallQual"

In [60]:
dataset = data[:,0:2]
matriz_x = np.hstack((dataset[:,1].reshape(dataset[:,1].size,1),np.ones((dataset[:,1].size,1))))

In [ ]:
def h(matriz_x, parametros_entrenables):
    return tf.linalg.matmul(matriz_x,parametros_entrenables)

def g(h, y):
    return tf.subtract(h,y)
def costo(g):
    return 1/2*tf.reduce_mean(tf.math.square(g))

In [59]:
tf.reset_default_graph()
g = tf.Graph()
with g.as_default():
    parametros_entrenables = tf.get_variable(name="Parametros_Entrenables", dtype=tf.float32, #shape=[2,1], 
                                initializer=[[0.5],[1.0]])
    input_data = tf.placeholder(dtype=tf.float32,shape=matriz_x.shape,name="Input_Data")
    
    
    tf.print(input_data)

    with tf.name_scope("Prediccion"):
        y_pred = h(input_data,parametros_entrenables)
    with tf.name_scope("Error"):
        error 
    with tf.train.MonitoredSession() as session:
        feed_dict = {input_data:matriz_x}
        
        session.run(y_pred,feed_dict=feed_dict)



INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[[11.5]
 [10. ]
 [11.5]
 ...
 [11.5]
 [ 8.5]
 [ 8.5]]
